In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("../dataset/Train_Data.csv") 
data.head()

,age,sex,bmi,smoker,region,children,charges
0,21.000000,male,25.745000,no,northeast,2,3279.868550
1,36.976978,female,25.744165,yes,southeast,3,21454.494239
2,18.000000,male,30.030000,no,southeast,1,1720.353700
3,37.000000,male,30.676891,no,northeast,3,6801.437542
4,58.000000,male,32.010000,no,southeast,1,11946.625900


In [6]:
data.shape

(3630, 7)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3630 entries, 0 to 3629
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       3630 non-null   float64
 1   sex       3630 non-null   object 
 2   bmi       3630 non-null   float64
 3   smoker    3630 non-null   object 
 4   region    3630 non-null   object 
 5   children  3630 non-null   int64  
 6   charges   3630 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 198.6+ KB


In [11]:
categorical_variables = data.select_dtypes(include='object').columns
categorical_variables

Index(['sex', 'smoker', 'region'], dtype='object')

In [10]:
numarical_variables = data.select_dtypes(include = ['float64','int64']).columns
numarical_variables

Index(['age', 'bmi', 'children', 'charges'], dtype='object')